# 1、create_sql_query_chain


举例1

In [2]:
from langchain_community.utilities import SQLDatabase

# 连接 MySQL 数据库
db_user = "root"
db_password = "123456"  #根据自己的密码填写
db_host = "127.0.0.1"
db_port = "3306"
db_name = "langchains_db"

# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")
print("哪种数据库：", db.dialect)
print("获取数据表：", db.get_usable_table_names())

# 执行查询
res = db.run("SELECT count(*) FROM employees;")

print("查询结果：", res)

哪种数据库： mysql
获取数据表： ['employees']
查询结果： [(1,)]


举例2：Chain的使用

In [3]:
from langchain_community.utilities import SQLDatabase
from langchain_openai import ChatOpenAI
from langchain.chains.sql_database.query import create_sql_query_chain

# 1、获取 mysql 数据库的连接
# 测试连接本地的 mysql 数据库
db_user = "root"
db_password = "123456"  #根据自己的密码填写
db_host = "127.0.0.1"
db_port = "3306"
db_name = "langchains_db"
# mysql+pymysql://用户名:密码@ip地址:端口号/数据库名
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

# 2、获取大预言模型
import os
import dotenv

# 加载环境变量
dotenv.load_dotenv()

# 环境变量进行赋值
os.environ["OPENAI_BASE_URL"] = os.getenv("OPENAI_BASE_URL")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 3、创建 create_sql_query_chain 实例
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there?"})

print(response)

SQLQuery: SELECT COUNT(`id`) AS `employee_count` FROM `employees`;


# 2、create_stuff_documents_chain（了解）

In [4]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# 定义提示词模板
prompt = PromptTemplate.from_template("""
基于如下文档{docs}中说，香蕉是什么颜色的？
""")

# 创建链
llm = ChatOpenAI(model="gpt-4o-mini")
chain = create_stuff_documents_chain(llm, prompt, document_variable_name="docs")

# 文档输入
docs = [
    Document(
        page_content="苹果，学名Malus pumila Mill.，别称西洋苹果、柰，属于蔷薇科苹果属的植物。苹果是全球最广泛种植和销售的水果之一，有悠久的栽培历史和广泛的分布范围。苹果的原始种群主要起源于中亚的天山山脉附近，尤其是现代哈萨克斯坦的阿拉木图地区，提供了所有现代苹果品种的基因库。苹果通过早期的贸易路线，如丝绸之路，从中亚向外扩散到全球各地。"
    ),
    Document(
        page_content="香蕉是白色的水果，主要产自热带地区。"
    ),
    Document(
        page_content="蓝莓是蓝色的浆果，含有抗氧化物质。"
    )
]

# 执行摘要
chain.invoke({"docs": docs})

'香蕉是白色的水果。'